In [ ]:
import json
file = []
with open('/scratch/cse/btech/cs1200448/MatLlama/ft_ds/train_ft.json', 'r') as f:
    for line in f:
        json_line = json.loads(line)
        file.append(json_line)

In [ ]:
len(file)

In [ ]:
import torch
import json
import numpy as np
from tqdm import tqdm
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
squad = json.load(open('../finetune/dataset/squadv2/squad_dev.json', 'r'))
# hellaswag = json.load(open('../finetune/dataset/hellaswag/hellaswag_val.jsonl', 'r'))
hellaswag = []
with open('../finetune/dataset/hellaswag/hellaswag_val.jsonl', 'r') as f:
    for line in f:
        json_line = json.loads(line)
        hellaswag.append(json_line)

In [ ]:
np.random.shuffle(hellaswag)

In [ ]:
hellaswag_dev = hellaswag[:1000]

In [ ]:
hellaswag_val = []
for i in hellaswag_dev:
    example = {'task':'completion', 'dataset':'hellaswag', 'input':i['ctx'], 'ind':i['ind'], 'options':i['endings'], 'answer':i['label']}
    hellaswag_val.append(example)

In [ ]:
squad_val = []
shot = 2
num_qn_per_ctx = 1
for topic in squad['data']:
    title = topic['title']
    for paragraph in topic['paragraphs']:
        ctx = paragraph['context']
        qas = [i for i in paragraph['qas'] if len(i['answers']) != 0]
        if len(qas) < shot+1:
            continue
        common_prompt = f'Title: {title}\n\nBackground: {ctx}\n\n'
        for which_shot in range(shot):
            common_prompt+= f'Q: {qas[which_shot]["question"]}\n\nA: {qas[which_shot]["answers"][0]["text"]}\n\n'
        for question in qas[shot:num_qn_per_ctx+shot]:
            answers = list(set([ans['text'] for ans in question['answers']]))
            example = {'task':'qna', 'dataset':'squadv2', 'qid':question['id'], 'valid_answers':answers}
            example['input'] = common_prompt + 'Q: ' + question['question'] + '\n\nA: '
            squad_val.append(example)

In [ ]:
matsci = json.load(open('/home/cse/btech/cs1200448/MatLlama/finetune/custom-finetuning-data/val_ft.json', 'r'))

In [ ]:
types = {}
for i in matsci:
    if (i['task'], i['dataset']) not in types:
        types[(i['task'], i['dataset'])] = [i]
    else:
        types[(i['task'], i['dataset'])].append(i)
matsci_val = {i:[] for i in types.keys()}

for type, qns in types.items():
    if type == ('pc', 'glass_non_glass'):
        shot = 2
        common_prompt = "Is the following sentence related to inorganic glass research?\n\n"
        for which_shot in range(shot):
            common_prompt+= f'Sentence: {qns[which_shot]["input"]}\n\nA: {qns[which_shot]["output"]}\n\n'
        for qn in qns[shot:]:
            example = {'task':type[0], 'dataset':type[1]}
            example['output'] = qn['output']
            example['input'] = common_prompt + 'Sentence: ' + qn['input'] + '\n\nA: '
            matsci_val[type].append(example)
            
    if type == ('sc', 'sofc_sent'):
        shot = 2
        common_prompt = "Is the following sentence related to solid oxide fuel cell research?\n\n"
        for which_shot in range(shot):
            common_prompt+= f'Sentence: {qns[which_shot]["input"]}\n\nA: {qns[which_shot]["output"]}\n\n'
        for qn in qns[shot:]:
            example = {'task':type[0], 'dataset':type[1]}
            example['output'] = qn['output']
            example['input'] = common_prompt + 'Sentence: ' + qn['input'] + '\n\nA: '
            matsci_val[type].append(example)
            
    if type == ('re', 'sc_comics'):
        shot = 2
        common_prompt = "Extract the relation between the entities from the context.\n\nOptions : condition, equivalent, target.\n\n"
        for which_shot in range(shot):
            input = qns[which_shot]["input"]
            output = qns[which_shot]["output"]
            input = input.split('\n')
            e1 = input[0].split("ENTITY1: ")[1]
            e2 = input[1].split("ENTITY2: ")[1]
            assert len(input) == 3
            common_prompt+= f'{input[2]}\nQ: What is the relation between {e1} and {e2}?\n\nA: {output}\n\n'
        for qn in qns[shot:]:
            if _ in shots:
                continue
            input = qn["input"]
            output = qn["output"]
            input = input.split('\n')
            e1 = input[0].split("ENTITY1: ")[1]
            e2 = input[1].split("ENTITY2: ")[1]
            assert len(input) == 3
            example = {'task':type[0], 'dataset':type[1]}
            example['output'] = output
            example['input'] = common_prompt + f'{input[2]}\nQ: What is the relation between {e1} and {e2}?\n\nA: '
            matsci_val[type].append(example)

    if type == ('re', 'structured_re'):
        common_prompt = "Extract the relation between the entities from the context.\n\nOptions : coulombic efficiency, capacity, conductivity, voltage, energy.\n\n"
        shots = [0,2]
        for which_shot in shots:
            input = qns[which_shot]["input"]
            output = qns[which_shot]["output"]
            input = input.split('\n')
            e1 = input[0].split("ENTITY1: ")[1]
            e2 = input[1].split("ENTITY2: ")[1]
            assert len(input) == 3
            common_prompt+= f'{input[2]}\nQ: What is the relation between {e1} and {e2}?\n\nA: {output}\n\n'
        for _, qn in enumerate(qns):
            if _ in shots:
                continue
            input = qn["input"]
            output = qn["output"]
            input = input.split('\n')
            e1 = input[0].split("ENTITY1: ")[1]
            e2 = input[1].split("ENTITY2: ")[1]
            assert len(input) == 3
            example = {'task':type[0], 'dataset':type[1]}
            example['output'] = output
            example['input'] = common_prompt + f'{input[2]}\nQ: What is the relation between {e1} and {e2}?\n\nA: '
            matsci_val[type].append(example)

    if type == ('ee', 'sc_comics'):
        common_prompt = "Extract the roles in doping for the given argument.\n\n"
        shots = [1]
        for which_shot in shots:
            input = qns[which_shot]["input"]
            output = qns[which_shot]["output"]
            input = input.split('\n')
            assert len(input) == 4
            output = output.split('\n')
            output = [i.split(' : ') for i in output]
            for otp in output:
                common_prompt+= f'{input[3]}\nQ: Is {otp[0]} the site or the dopant?\n\nA: {otp[1]}\n\n'
        for _, qn in enumerate(qns):
            if _ in shots:
                continue
            input = qn["input"]
            output = qn["output"]
            input = input.split('\n')
            assert len(input) == 4
            output = output.split('\n')
            output = [i.split(' : ') for i in output]
            # print(_,input,output)
            for otp in output:
                example = {'task':type[0], 'dataset':type[1]}
                example['output'] = otp[1]
                example['input'] = common_prompt + f'{input[3]}\nQ: Is {otp[0]} the site or the dopant?\n\nA: '
                matsci_val[type].append(example)

    if type == ('sar', 'synthesis_actions'):
        common_prompt = "Extract the synthesis action for the Verb in the given sentence.\n\nOptions : cooling, heating, mixing, non-altering, purification, reaction, shaping, starting.\n\n"
        shots = [1]
        for which_shot in shots:
            input = qns[which_shot]["input"]
            output = qns[which_shot]["output"]
            input = input.split('\n')
            output = output.split('\n')
            output = [i.split(' : ') for i in output]
            input = "\n".join([i for _,i in enumerate(input) if _ != 0])
            for otp in output:
                common_prompt+= f'{input}\nQ: What is the synthesis action for {otp[0]}?\n\nA: {otp[1]}\n\n'
        for _, qn in enumerate(qns):
            if _ in shots:
                continue
            input = qn["input"]
            output = qn["output"]
            input = input.split('\n')
            input = "\n".join([i for _,i in enumerate(input) if _ != 0])
            output = output.split('\n')
            output = [i.split(' : ') for i in output]
            # print(_,input,output)
            for otp in output:
                example = {'task':type[0], 'dataset':type[1]}
                example['output'] = otp[1]
                example['input'] = common_prompt + f'{input}\nQ: What is the synthesis action for {otp[0]}?\n\nA: '
                matsci_val[type].append(example)

    if type == ('ner', 'sc_comics'):
        num_examples = 500
        which_entities = ['material', 'process', 'element', 'property']
        shots = [0,5]
        # material, doping, sc, value, process, characterization, element, property, main
        for entity in which_entities:
            common_prompt = f"Extract the {entity} in the following sentences. If there are none, respond with 'None'. If there are multiple, separate them with ';'.\n\n"
            for which_shot in shots:
                input = qns[which_shot]["input"]
                output = qns[which_shot]["output"]
                input = input.split('\n')[-1]
                output = output.split('\n')
                output = [i.split(' : ') for i in output]
                output = [i for i in output if i[1] == entity]
                output = ";".join(list(set([i[0] for i in output])))
                common_prompt+= f'{input}\nQ: What are the {entity}?\n\nA: {"None" if len(output) == 0 else output}\n\n'
            for _, qn in enumerate(qns[:num_examples]):
                if _ in shots:
                    continue
                input = qn["input"]
                output = qn["output"]
                input = input.split('\n')[-1]
                output = output.split('\n')
                output = [i.split(' : ') for i in output]
                output = [i for i in output if i[1] == entity]
                if len(output) == 0:
                    output = "None"    
                else:
                    output = ";".join(list(set([i[0] for i in output])))
                example = {'task':type[0], 'dataset':type[1]}
                example['output'] = output
                example['input'] = common_prompt + f'{input}\nQ: What are the {entity}?\n\nA: '
                matsci_val[type].append(example)

In [ ]:
print(*[[j,len(i)] for j,i in matsci_val.items()])
print(len(hellaswag_val))
print(len(squad_val))

In [ ]:
zshot_val = squad_val
# zshot_val = hellaswag_val + squad_val
for i,j in matsci_val.items():
    zshot_val += j

In [ ]:
# json.dump(zshot_val, open('../finetune/custom-finetuning-data/Kshot-val.json', 'w'), indent = 1)

In [ ]:
import json
kshot = json.load(open('../finetune/custom-finetuning-data/Kshot-val.json', 'r'))

In [ ]:
a = [i for i in kshot if i['task'] == 'sc']

In [ ]:
b = sum([1 for i in a if i['output'] == 'yes'])
c = sum([1 for i in a if i['output'] == 'no'])

In [ ]:
c

In [ ]:
# for item in matsci:
#     if item['task']=='ner' and item['dataset']=='matscholar':
#         tasks = item['input'].split('SENTENCE:')[0].split('WORDS:')[1].split(',')
#         sentence = item['input'].split('SENTENCE:')[1]
#         outputs = item['output']
#         options = ['b-mat', 'i-mat', 'b-spl', 'i-spl', 'b-dsc', 'i-dsc', 'b-pro', 'i-pro', 'b-apl', 'i-apl', 'b-smt', 'i-smt', 'b-cmt', 'i-cmt']
#         sample = 'Context: '
#         sample += sentence + '\n\n'
#         sample += 'Options: '
#         sample += ', '.join(options) + '\n\n'
#         tasks = list(map(lambda x: x.replace('\n', '').strip(), tasks))
#         if len(tasks)<=2:
#             continue
#         outputs = outputs.split('\n')
#         output_dict = dict()
#         for output in outputs:
#             output_dict[output.split(':')[0].strip()] = output.split(':')[1].strip()

#         try:
#             sample += f'Q: Named entity for {tasks[0]}\n\nA: {output_dict[tasks[0]]}\n\n'
#             sample += f'Q: Named entity for {tasks[1]}\n\nA: {output_dict[tasks[1]]}\n\n'
        
#             for task in tasks[2:]:  
#                 try:
#                     dataset.append({
#                         'input':sample + f'Q: Named entity for {task}\n\nA: \n\n',
#                         'output': output_dict[task],
#                         'task': 'ner',
#                         'dataset': 'matscholar'
#                    })
#                 except:
#                     continue
#         except:
#             continue
#     if item['task']=='ner' and item['dataset']=='sofc_token':
#         tasks = item['input'].split('SENTENCE:')[0].split('WORDS:')[1].split(',')
#         sentence = item['input'].split('SENTENCE:')[1]
#         outputs = item['output']
#         options = ['b-mat', 'i-mat', 'b-spl', 'i-spl', 'b-dsc', 'i-dsc', 'b-pro', 'i-pro', 'b-apl', 'i-apl', 'b-smt', 'i-smt', 'b-cmt', 'i-cmt']
#         sample = 'Context: '
#         sample += sentence + '\n\n'
#         sample += 'Options: '
#         sample += ', '.join(options) + '\n\n'
#         tasks = list(map(lambda x: x.replace('\n', '').strip(), tasks))
#         if len(tasks)<=2:
#             continue
#         outputs = outputs.split('\n')
#         output_dict = dict()
#         for output in outputs:
#             output_dict[output.split(':')[0].strip()] = output.split(':')[1].strip()

#         try:
#             sample += f'Q: Named entity for {tasks[0]}\n\nA: {output_dict[tasks[0]]}\n\n'
#             sample += f'Q: Named entity for {tasks[1]}\n\nA: {output_dict[tasks[1]]}\n\n'
        
#             for task in tasks[2:]:  
#                 try:
#                     dataset.append({
#                         'input':sample + f'Q: Named entity for {task}\n\nA: \n\n',
#                         'output': output_dict[task],
#                         'task': 'ner',
#                         'dataset': 'sofc_token'
#                    })
#                 except:
#                     continue
#         except:
#             continue

#     if item['task']=='pc':
#         sample = 'Context: '
#         sample += item['input'] + '\n\n'
#         sample += 'Q: Is the above paragraph related to inorganic glass research?\n\nA: '
#         dataset.append({
#             'input': sample,
#             'output': item['output'],
#             'task': 'pc',
#             'dataset': 'glass_non_glass'
#         })
#     if item['task']=='sc':
#         sample = 'Context: '
#         sample += item['input'] + '\n\n'
#         sample += 'Q: Is this paragraph related to solid oxide fuel cell (SOFC) research?\n\nA: '
#         dataset.append({
#             'input': sample,
#             'output': item['output'],
#             'task': 'sc',
#             'dataset': 'sofc_sent'
#         })

In [ ]:
import json
import random

random.seed(42)

with open('/home/cse/btech/cs1200448/MatLlama/finetune/dataset/squadv2/squad_train.json', 'r') as f:
	data = json.load(f)

ds = []
for item in data['data']:
	for para in item['paragraphs']:
		context = para['context']
		try:
			qas = list(map(lambda x: (x['question'], x['answers'][0]['text']), para['qas']))
			ds.append([context, qas])
		except:
			continue

SYSTEM = "You will be given a text and you have to answer the question that follows it using information from the text."

train_ds = []
val_ds = []
for context, qas in ds:	
	for q, a in qas:
		temp = random.random()
		if temp < 0.023:
			train_ds.append({'system': SYSTEM,
					 'question': context+' '+q,
					 'answer': a})
		elif 0.023 < temp < 0.046:
			val_ds.append({'system': SYSTEM,
				       'question': context+' '+q,
				       'answer': a})

with open('/scratch/cse/btech/cs1200448/MatLlama/ft_ds/train_ft.jsonl', 'a') as f:
	for doc in train_ds:
		f.write(json.dumps(doc)+'\n')

with open('/scratch/cse/btech/cs1200448/MatLlama/ft_ds/val_ft.jsonl', 'a') as f:
    for doc in val_ds:
            f.write(json.dumps(doc)+'\n')
